In [1]:
import data_center as dc
import datetime

In [2]:
p = dc.data_center()
ubikedb = p.connect2db()
ubikecol = ubikedb["new_ubike_col"]

In [50]:
start = datetime.datetime(2021, 3, 31, 0, 0, 0)
tdelta = datetime.timedelta(hours=12)
#end = datetime.datetime(2021, 4, 1, 00, 00, 00)
end = start + tdelta
#d = ubikecol.find( {'sno': 1, 'mday': {'$lt': end, '$gte': start}}).limit(3)
#query = {'sno': 1, 'mday': {'$lt': end, '$gte': start},}
#query = {'mday': {'$lt': end, '$gte': start}}
query = {'sno': 1}
#query = {}

In [51]:
ret = p.db_iterate_find(ubikecol,query)

query_cnt: 10000
query_cnt: 10000
query_cnt: 1222


In [56]:
ubike_df = ret[['sno','mday','sna','bemp','sbi']]

In [57]:
ubike_df

,sno,mday,sna,bemp,sbi
0,1,2021-03-27 18:01:39,捷運市政府站(3號出口),45,135
1,1,2021-03-27 18:02:39,捷運市政府站(3號出口),49,131
2,1,2021-03-27 18:03:38,捷運市政府站(3號出口),55,125
3,1,2021-03-27 18:04:38,捷運市政府站(3號出口),54,126
4,1,2021-03-27 18:05:39,捷運市政府站(3號出口),56,124
...,...,...,...,...,...
1217,1,2021-04-12 21:22:42,捷運市政府站(3號出口),175,5
1218,1,2021-04-12 21:23:42,捷運市政府站(3號出口),178,2
1219,1,2021-04-12 21:24:42,捷運市政府站(3號出口),179,1
1220,1,2021-04-12 21:25:42,捷運市政府站(3號出口),179,1


In [69]:
import pandas as pd

In [40]:
import data_preprocessor as dp

In [19]:
weathercol = ubikedb["new_weather_col"]
weather2col = ubikedb["new_weather2_col"]

In [20]:
#query = {'time': {'$lt': end, '$gte': start}}
ret1 = p.db_iterate_find(weathercol,query)
ret2 = p.db_iterate_find(weather2col,query)

query_cnt: 2915
query_cnt: 776


In [123]:
import calendar

def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return date(year, month, day)

def cal_dist(coor1, coor2):
    return (coor1[0]-coor2[0])**2+(coor1[1]-coor2[1])**2

print(ret1)
print(ret2)

                          _id stationId locationName        lat         lon  \
0    607a9a46459724b1afa13405    C0AH70           松山  25.050492  121.542203   
1    607a9a46459724b1afa13406    C0AC80           文山  25.004083  121.567528   
2    607a9a46459724b1afa13407    C0A9F0           內湖  25.081206  121.567222   
3    607a9a46459724b1afa13408    C0A9C0           天母  25.119253  121.528939   
4    607a9a46459724b1afa13409    C0A9E0           士林  25.092031  121.494792   
..                        ...       ...          ...        ...         ...   
98   607a9a47459724b1afa13467    C0A9E0           士林  25.092031  121.494792   
99   607a9a47459724b1afa13468    C0A980           社子  25.111389  121.461389   
100  607a9a47459724b1afa13469    C0AC40          大屯山  25.177333  121.514167   
101  607a9a47459724b1afa1346a    C0AH40           平等  25.130925  121.568856   
102  607a9a47459724b1afa1346b    C0AI40           石牌  25.117336  121.504825   

                   time   WDIR  WDSE  TEMP  HUMD   

In [131]:
try:
    # resample
    ubike_df = ubike_df.set_index(ubike_df.mday)
    grouped_ubike_df = ubike_df.resample('min').mean()

    #remove data not act/vaild
    get_data = grouped_ubike_df[grouped_ubike_df['act'] == 0]
    grouped_ubike_df = grouped_ubike_df.drop(get_data.index)
    
    #get weahter data
    processed_wdata = ret1
    processed_wdata1 = ret2

    # seperate weather station
    wstation = processed_wdata.groupby(['stationId'])
    station_1 = wstation.first()[['locationName','lat','lon']]
    wstation = processed_wdata1.groupby(['stationId'])
    station_2 = wstation.first()[['locationName','lat','lon']]

    print('start to process weather data')
    ubike_location = [ubike_df[ubike_df['sno'] == 150].iloc[0]['lat'], ubike_df[ubike_df['sno'] == 150].iloc[0]['lng']]
    
    # find nearest staton, get coordinate of ubike station
    print('start to process weather coordinates')
    dist_min = 1000
    nearest_station1 = 0

    for index, row in station_1.iterrows():
        station_loc = [row['lat'], row['lon']]
        distance = cal_dist(station_loc, ubike_location)
        if(distance < dist_min):
            dist_min = distance
            nearest_station1 = index
            
    dist_min = 1000
    nearest_station2 = 0

    for index, row in station_2.iterrows():
        station_loc = [row['lat'], row['lon']]
        distance = cal_dist(station_loc, ubike_location)
        if(distance < dist_min):
            dist_min = distance
            nearest_station2 = index
    

    # group all weather data by station
    wdata1 = processed_wdata.loc[processed_wdata.stationId == nearest_station1]
    wdata2 = processed_wdata1.loc[processed_wdata1.stationId == nearest_station2]

    
    wdata1.set_index(keys = ['time'], inplace = True)
    wdata2.set_index(keys=['time'], inplace = True)
    wdata1.index = wdata1.index - pd.Timedelta(hours=0.5)
    wdata1 = wdata1.resample('H').fillna("pad")
    wdata2 = wdata2.resample('H').fillna("pad")
    #ubike resample to 1 hour
    grouped_ubike_df = grouped_ubike_df.resample("H").mean()
    
    print('resample group data finished')

    #print(grouped_ubike_df.columns)
    print(wdata1.columns)
    print(wdata2.columns)
  

    
    data_group = grouped_ubike_df.drop(columns=['lat','lng','act'])
    wdata1_short = wdata1.drop(columns=['_id', 'stationId', 'locationName', 'lat', 'lon','WSGust', 'WDGust'])
    wdata2_short = wdata2.drop(columns=['stationId', 'locationName', 'lat', 'lon','WDIR', 'WDSE', 'TEMP', 'HUMD', 'PRES','24R'])
    
    print(wdata2.columns)
    weather_group = pd.merge(wdata1_short,wdata2_short,left_index=True, right_index=True)
    weather_group = weather_group.drop(columns=['_id'])
    weather_group = weather_group.sort_index()             
    group_data =pd.merge(data_group,weather_group,left_index=True, right_index=True)
    print(group_data)

except Exception as e:
    print(e)


start to process weather data
start to process weather coordinates
resample group data finished
Index(['_id', 'stationId', 'locationName', 'lat', 'lon', 'WDIR', 'WDSE',
       'TEMP', 'HUMD', 'PRES', 'H_24R', 'WSGust', 'WDGust'],
      dtype='object')
Index(['_id', 'stationId', 'locationName', 'lat', 'lon', 'WDIR', 'WDSE',
       'TEMP', 'HUMD', 'PRES', '24R', 'WSGust', 'WDGust', 'UVI', 'PrecpHour',
       'Visb'],
      dtype='object')
Index(['_id', 'stationId', 'locationName', 'lat', 'lon', 'WDIR', 'WDSE',
       'TEMP', 'HUMD', 'PRES', '24R', 'WSGust', 'WDGust', 'UVI', 'PrecpHour',
       'Visb'],
      dtype='object')
                       sno   tot        sbi       bemp   WDIR  WDSE  TEMP  \
2021-03-31 00:00:00  150.0  38.0  12.017857  25.982143   70.0   1.0  21.1   
2021-03-31 01:00:00  150.0  38.0  12.526316  25.473684   97.0   0.6  20.9   
2021-03-31 02:00:00  150.0  38.0  13.877193  24.122807    0.0   0.2  21.1   
2021-03-31 03:00:00  150.0  38.0  12.438596  25.561404    0.0 

In [130]:
group_data

,tot,sbi,bemp,WDIR,WDSE,TEMP,HUMD,PRES,H_24R,WSGust,WDGust,UVI,PrecpHour,Visb
2021-03-31 00:00:00,38.0,12.017857,25.982143,70.0,1.0,21.1,0.95,1005.0,0.0,3.0,30.0,0.00,0.0,21-30
2021-03-31 01:00:00,38.0,12.526316,25.473684,97.0,0.6,20.9,0.95,1004.6,0.0,4.7,130.0,0.00,0.0,16-20
2021-03-31 02:00:00,38.0,13.877193,24.122807,0.0,0.2,21.1,0.95,1004.5,0.5,3.5,360.0,0.00,0.0,16-20
2021-03-31 03:00:00,38.0,12.438596,25.561404,0.0,0.2,21.1,0.95,1004.5,0.5,2.8,20.0,0.00,0.0,3-6
2021-03-31 04:00:00,38.0,13.981481,24.018519,0.0,0.1,20.7,0.96,1003.4,0.5,3.9,100.0,0.00,0.0,16-20
2021-03-31 05:00:00,38.0,14.750000,23.250000,346.0,0.6,20.6,0.96,1003.0,1.0,2.8,30.0,0.00,0.0,16-20
2021-03-31 06:00:00,38.0,14.563636,23.436364,0.0,0.0,20.5,0.96,1002.9,1.0,2.4,100.0,0.04,0.0,21-30
2021-03-31 07:00:00,38.0,22.072727,15.927273,0.0,0.0,21.0,0.97,1003.0,1.0,2.1,100.0,0.49,0.3,16-20
2021-03-31 08:00:00,38.0,27.444444,10.555556,0.0,0.0,22.4,0.96,1003.4,1.0,2.4,210.0,1.80,1.1,11-15
2021-03-31 09:00:00,38.0,32.821429,5.160714,0.0,0.2,24.5,0.88,1004.2,1.0,2.6,170.0,3.64,2.1,11-15


In [21]:
group_data.Visb.unique()

NameError: name 'group_data' is not defined

In [118]:
wdata1

,_id,stationId,locationName,lat,lon,WDIR,WDSE,TEMP,HUMD,PRES,24R,WSGust,WDGust,UVI,PrecpHour,Visb
time,,,,,,,,,,,,,,,,
2021-03-31 00:00:00,607a9a645a69b5e37c506fa5,466920,臺北,25.03941,121.506676,100.0,1.7,22.5,0.95,1005.7,0.0,3.0,30.0,0.00,0.0,21-30
2021-03-31 01:00:00,607a9a645a69b5e37c506fa7,466920,臺北,25.03941,121.506676,50.0,1.0,22.6,0.96,1005.7,0.0,4.7,130.0,0.00,0.0,16-20
2021-03-31 02:00:00,607a9a645a69b5e37c506fa9,466920,臺北,25.03941,121.506676,80.0,0.7,22.7,0.95,1005.5,0.0,3.5,360.0,0.00,0.0,16-20
2021-03-31 03:00:00,607a9a645a69b5e37c506fab,466920,臺北,25.03941,121.506676,100.0,1.6,21.5,0.98,1004.8,0.0,2.8,20.0,0.00,0.0,3-6
2021-03-31 04:00:00,607a9a645a69b5e37c506fad,466920,臺北,25.03941,121.506676,40.0,0.7,21.4,0.98,1004.3,0.0,3.9,100.0,0.00,0.0,16-20
2021-03-31 05:00:00,607a9a645a69b5e37c506faf,466920,臺北,25.03941,121.506676,80.0,1.1,21.3,0.97,1004.1,0.0,2.8,30.0,0.00,0.0,16-20
2021-03-31 06:00:00,607a9a645a69b5e37c506fb1,466920,臺北,25.03941,121.506676,120.0,0.8,21.6,0.96,1003.8,0.0,2.4,100.0,0.04,0.0,21-30
2021-03-31 07:00:00,607a9a645a69b5e37c506fb3,466920,臺北,25.03941,121.506676,230.0,0.6,22.8,0.95,1004.2,0.0,2.1,100.0,0.49,0.3,16-20
2021-03-31 08:00:00,607a9a645a69b5e37c506fb5,466920,臺北,25.03941,121.506676,180.0,1.2,24.9,0.92,1004.8,0.0,2.4,210.0,1.80,1.1,11-15


In [112]:
wdata2

,_id,stationId,locationName,lat,lon,WDIR,WDSE,TEMP,HUMD,PRES,H_24R,WSGust,WDGust
time,,,,,,,,,,,,,
2021-03-31 00:00:00,607a9a46459724b1afa13406,C0AC80,文山,25.004083,121.567528,70.0,1.0,21.1,0.95,1005.0,0.0,-99.0,-99.0
2021-03-31 01:00:00,607a9a46459724b1afa1340f,C0AC80,文山,25.004083,121.567528,97.0,0.6,20.9,0.95,1004.6,0.0,-99.0,-99.0
2021-03-31 02:00:00,607a9a46459724b1afa13418,C0AC80,文山,25.004083,121.567528,0.0,0.2,21.1,0.95,1004.5,0.5,-99.0,-99.0
2021-03-31 04:00:00,607a9a46459724b1afa13426,C0AC80,文山,25.004083,121.567528,0.0,0.1,20.7,0.96,1003.4,0.5,-99.0,-99.0
2021-03-31 05:00:00,607a9a46459724b1afa1342f,C0AC80,文山,25.004083,121.567528,346.0,0.6,20.6,0.96,1003.0,1.0,-99.0,-99.0
2021-03-31 06:00:00,607a9a46459724b1afa13438,C0AC80,文山,25.004083,121.567528,0.0,0.0,20.5,0.96,1002.9,1.0,-99.0,-99.0
2021-03-31 07:00:00,607a9a46459724b1afa13441,C0AC80,文山,25.004083,121.567528,0.0,0.0,21.0,0.97,1003.0,1.0,-99.0,-99.0
2021-03-31 08:00:00,607a9a47459724b1afa1344a,C0AC80,文山,25.004083,121.567528,0.0,0.0,22.4,0.96,1003.4,1.0,-99.0,-99.0
2021-03-31 09:00:00,607a9a47459724b1afa13453,C0AC80,文山,25.004083,121.567528,0.0,0.2,24.5,0.88,1004.2,1.0,-99.0,-99.0


In [27]:
import os
import pandas as pd
filepath = "E:/csvfile/parsed/bemp/"
filesinpath = os.listdir(filepath)
na_list = []
for f in sorted(filesinpath): #for each file, ran model
    #print("file name:", f)
    df = pd.read_csv(filepath + f)

    #df = df.dropna()
    print(df.columns)
    print(df.describe())
    df['time'] = pd.to_datetime(df['time'], format='%Y/%m/%d %H%M%S', errors='ignore')
    df = df.set_index(pd.DatetimeIndex(df['time']))
    X = df.drop(columns=['time.1','y_bemp'])
    #na_ex = X.isnull().values.any()
    #na_list = na_list + [na_ex]
    break
    


Index(['time', 'time.1', 'bemp', 'sno', 'HUMD', 'H_24R', 'PRES', 'TEMP',
       'WDIR', 'WDSE', 'PrecpHour', 'UVI', 'Visb', 'WDGust', 'WSGust',
       'weekday', 'hours', 'holiday', 'bemp_1h', 'bemp_2h', 'bemp_3h',
       'bemp_4h', 'bemp_5h', 'bemp_6h', 'bemp_7h', 'bemp_8h', 'bemp_9h',
       'bemp_10h', 'bemp_11h', 'bemp_12h', 'bemp_1d', 'bemp_2d', 'bemp_3d',
       'bemp_4d', 'bemp_5d', 'bemp_6d', 'bemp_7d', 'predict_hour', 'y_bemp'],
      dtype='object')
                bemp       sno           HUMD          H_24R           PRES  \
count  101854.000000  101854.0  101854.000000  101854.000000  101854.000000   
mean      100.192403       1.0      79.229117       0.186507    1007.833506   
std        42.532843       0.0      13.934679       1.448780       6.713548   
min         5.000000       1.0      28.000000       0.000000     984.300000   
25%        66.000000       1.0      69.000000       0.000000    1002.600000   
50%        97.000000       1.0      83.000000       0.000000  

In [13]:
print(df.UVI.describe())

count    101802.000000
mean          4.258880
std           9.686738
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          70.000000
Name: UVI, dtype: float64


In [14]:
df.UVI.unique()

array([ 0. , 10. , 20. , 19. ,  2. ,  6. , 18. ,  5. ,  3. ,  2.5, 11. ,
       16. , 15. ,  9. , 12. , 25. , 24. , 30. , 27. ,  4. ,  8. ,  7. ,
       13. , 40. ,  3.5, 45. , 50. , 55. , 17. , 35. , 22. , 14. , 23. ,
        1. , 28. , 60. ,  1.5, 26. , 70. , 21. ])

In [23]:
ret1.describe()

,lat,lon,WDIR,WDSE,TEMP,HUMD,PRES,H_24R,WSGust,WDGust
count,2915.000000,2915.000000,2915.000000,2915.000000,2915.000000,2915.000000,2915.000000,2915.000000,2915.000000,2915.000000
mean,25.098327,121.527381,135.601029,1.756398,20.581475,0.770844,993.988302,0.183877,-82.226861,-69.783190
std,0.046663,0.035264,109.868036,1.551965,4.542281,0.143536,38.142970,1.947046,39.377324,74.528117
min,25.004083,121.461389,0.000000,0.000000,4.800000,0.190000,887.500000,-99.000000,-99.000000,-99.000000
25%,25.081206,121.504825,64.000000,0.600000,17.400000,0.675000,1003.400000,0.000000,-99.000000,-99.000000
50%,25.111389,121.528939,96.000000,1.400000,20.900000,0.780000,1010.200000,0.000000,-99.000000,-99.000000
75%,25.119253,121.567222,242.000000,2.500000,23.900000,0.885000,1015.400000,0.000000,-99.000000,-99.000000
max,25.177333,121.568856,360.000000,10.900000,31.400000,1.000000,1022.000000,3.500000,18.200000,355.000000


In [24]:
ret2.describe()

,lat,lon,WDIR,WDSE,TEMP,HUMD,PRES,24R,WSGust,WDGust,UVI,PrecpHour
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,25.111901,121.514096,155.106959,1.780412,18.540206,0.368299,452.832088,-0.081186,6.542784,163.158505,0.176817,1.703351
std,0.072537,0.007424,91.451832,6.378917,9.850815,7.158471,555.052908,6.254174,4.974625,98.396430,11.651895,6.872660
min,25.039410,121.506676,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000
25%,25.039410,121.506676,100.000000,1.200000,16.600000,0.810000,-99.000000,0.000000,3.900000,90.000000,0.000000,0.000000
50%,25.111901,121.514095,130.000000,2.000000,19.100000,0.940000,-99.000000,0.000000,6.300000,140.000000,0.010000,0.100000
75%,25.184391,121.521515,200.000000,2.900000,22.600000,0.980000,1010.950000,0.000000,9.000000,220.000000,1.635000,4.000000
max,25.184391,121.521515,360.000000,6.400000,30.400000,1.000000,1018.700000,5.500000,15.800000,360.000000,12.370000,10.100000


In [25]:
ret2.UVI.unique()

array([ 2.700e-01,  2.100e-01,  1.000e-02,  0.000e+00,  8.000e-02,
        1.200e-01,  3.100e-01,  7.900e-01,  2.060e+00,  2.240e+00,
        5.340e+00,  4.620e+00,  7.850e+00,  6.750e+00,  7.490e+00,
        7.790e+00,  3.470e+00,  6.570e+00,  6.440e+00,  5.510e+00,
        4.960e+00,  3.350e+00,  2.130e+00,  1.270e+00,  1.120e+00,
        2.500e-01,  2.400e-01,  1.500e-01,  9.800e-01,  5.800e-01,
        2.840e+00,  1.130e+00,  5.350e+00,  3.030e+00,  8.450e+00,
        6.860e+00,  9.020e+00,  8.690e+00, -9.900e+01,  5.440e+00,
        6.790e+00,  4.550e+00,  2.950e+00,  2.350e+00,  1.040e+00,
        8.200e-01,  1.700e-01,  1.300e-01,  2.000e-02,  7.000e-02,
        3.900e-01,  3.600e-01,  1.480e+00,  1.290e+00,  2.920e+00,
        1.650e+00,  4.410e+00,  1.860e+00,  4.160e+00,  2.600e+00,
        5.860e+00,  1.620e+00,  6.100e-01,  2.610e+00,  1.340e+00,
        1.540e+00,  9.400e-01,  2.900e-01,  2.200e-01,  1.100e-01,
        4.000e-02,  1.900e-01,  4.900e-01,  9.900e-01,  1.800e

In [26]:
ret2.Visb.unique()

array(['>30', '21-30', '16-20', '7-10', '11-15', '3-6', '無觀測', '<1', 2.6,
       2.3, 2.8, 2.9, 2.7, 1.2, 1.8, 2.0, 2.2, 2.1, 1.6, 1.1, 1.0, 1.3,
       2.5], dtype=object)

In [29]:
X.Visb.describe()

count    101854.000000
mean          0.510352
std           0.810529
min           0.000000
25%           0.000000
50%           0.000000
75%           0.770000
max           3.280000
Name: Visb, dtype: float64

In [30]:
ret2.Visb

0        >30
1      21-30
2        >30
3      21-30
4        >30
       ...  
771      >30
772    11-15
773    21-30
774      >30
775    16-20
Name: Visb, Length: 776, dtype: object

In [31]:
ret2.Visb.to_csv("UVI_observe.csv")  

In [32]:
for i in ret2.Visb:
    print(i)

>30
21-30
>30
21-30
>30
16-20
21-30
16-20
21-30
16-20
21-30
21-30
>30
21-30
>30
21-30
21-30
21-30
16-20
21-30
16-20
21-30
16-20
21-30
7-10
21-30
11-15
21-30
16-20
21-30
21-30
21-30
>30
16-20
>30
16-20
7-10
16-20
21-30
16-20
>30
16-20
>30
11-15
>30
11-15
>30
16-20
>30
16-20
>30
16-20
>30
11-15
>30
7-10
>30
7-10
>30
3-6
>30
7-10
21-30
3-6
7-10
7-10
3-6
7-10
3-6
3-6
3-6
7-10
11-15
7-10
16-20
3-6
11-15
7-10
>30
3-6
>30
3-6
>30
7-10
>30
7-10
無觀測
無觀測
16-20
7-10
21-30
7-10
16-20
11-15
11-15
7-10
7-10
7-10
7-10
7-10
3-6
11-15
3-6
7-10
3-6
7-10
3-6
3-6
7-10
3-6
11-15
3-6
7-10
3-6
7-10
3-6
16-20
7-10
11-15
7-10
<1
7-10
<1
7-10
2.6
7-10
3-6
7-10
16-20
11-15
7-10
11-15
3-6
16-20
7-10
16-20
7-10
16-20
2.3
11-15
>30
21-30
>30
21-30
>30
21-30
>30
21-30
>30
7-10
16-20
11-15
3-6
11-15
>30
21-30
3-6
21-30
3-6
21-30
2.8
16-20
3-6
16-20
7-10
3-6
2.9
16-20
3-6
16-20
3-6
21-30
<1
16-20
11-15
11-15
21-30
11-15
11-15
21-30
21-30
16-20
16-20
11-15
11-15
11-15
2.7
11-15
2.7
11-15
11-15
11-15
1.2
3-6
<1
7-10
<1


In [36]:
def update_uvi_category(df):
    UVI_catgory = ['>30', '21-30', '16-20', '11-15','7-10', '3-6', '1-2','<1','0']
    uvi_c = pd.Series(index = df.index,data=[None for _ in range(len(df.index))])
    uvi_c[df.UVI > 30] = '>30'
    idx = (df.UVI > 20) & (df.UVI <= 30)
    uvi_c[idx] = '20-30'
    idx = (df.UVI > 11) & (df.UVI <= 16)
    uvi_c[idx] = '11-15'
    idx = (df.UVI > 7) & (df.UVI <= 11)
    uvi_c[idx] = '7-10'
    idx = (df.UVI > 3) & (df.UVI <= 7)
    uvi_c[idx] = '3-6'
    idx = (df.UVI > 1) & (df.UVI <= 3)
    uvi_c[idx] = '1-2'
    uvi_c[df.UVI <= 1] = '1'
    uvi_c[df.UVI == 0] = '0'

    #print(uvi_c.value_counts())
    df.UVI = uvi_c


    return df


update_uvi_category(df)

,time,time.1,bemp,sno,HUMD,H_24R,PRES,TEMP,WDIR,WDSE,...,bemp_12h,bemp_1d,bemp_2d,bemp_3d,bemp_4d,bemp_5d,bemp_6d,bemp_7d,predict_hour,y_bemp
time,,,,,,,,,,,,,,,,,,,,,
2018-02-08 00:00:00,2018-02-08 00:00:00,2018-02-08 00:00:00,92.0,1.0,96.0,2.00,1016.8,11.0,0.0,0.0,...,86.0,132.0,95.0,63.0,106.0,119.0,72.0,48.0,1,92.0
2018-02-08 01:00:00,2018-02-08 01:00:00,2018-02-08 01:00:00,92.0,1.0,96.0,2.00,1016.8,11.0,71.0,0.7,...,86.0,137.0,99.0,64.0,107.0,115.0,77.0,50.0,1,92.0
2018-02-08 02:00:00,2018-02-08 02:00:00,2018-02-08 02:00:00,92.0,1.0,93.0,1.50,1016.7,11.0,82.0,1.2,...,87.0,140.0,102.0,65.0,110.0,117.0,85.0,52.0,1,93.0
2018-02-08 03:00:00,2018-02-08 03:00:00,2018-02-08 03:00:00,93.0,1.0,90.0,0.50,1016.5,10.8,79.0,1.4,...,89.0,145.0,104.0,67.0,111.0,118.0,88.0,54.0,1,94.0
2018-02-08 04:00:00,2018-02-08 04:00:00,2018-02-08 04:00:00,94.0,1.0,92.0,0.50,1016.5,10.5,86.0,1.3,...,91.0,149.0,105.0,68.0,112.0,119.0,90.0,55.0,1,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-29 08:00:00,2018-12-29 08:00:00,2018-12-29 08:00:00,69.0,1.0,87.0,0.00,1027.0,11.7,80.0,3.0,...,9.0,68.0,150.0,115.0,96.0,68.0,86.0,127.0,12,37.0
2018-12-29 09:00:00,2018-12-29 09:00:00,2018-12-29 09:00:00,102.0,1.0,83.0,0.50,1026.6,12.0,82.0,3.3,...,16.0,58.0,154.0,120.0,102.0,70.0,76.0,118.0,12,45.0
2018-12-29 10:00:00,2018-12-29 10:00:00,2018-12-29 10:00:00,91.0,1.0,79.0,0.25,1025.7,12.7,78.0,3.8,...,25.0,64.0,120.0,117.0,96.0,64.0,79.0,92.0,12,55.0


In [45]:
data_encoded=pd.get_dummies(data=df.UVI,drop_first=True)

In [47]:
data_encoded

,1
time,
2018-02-08 00:00:00,0
2018-02-08 01:00:00,0
2018-02-08 02:00:00,0
2018-02-08 03:00:00,0
2018-02-08 04:00:00,0
...,...
2018-12-29 08:00:00,0
2018-12-29 09:00:00,0
2018-12-29 10:00:00,0
